# Configure Sugar


In [ ]:
#| default_exp config

In [ ]:
#| hide
from nbdev.showdoc import *


# load env
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
#| export

import os
from sugar.web3 import normalize_address
from typing import Literal

In [ ]:
#| export

class SugarConfig:
    web3_provoider_uri: str = os.environ["WEB3_PROVIDER_URI"]
    lp_sugar_address: str = os.environ["LP_SUGAR_ADDRESS"]
    token_address: str = normalize_address(os.getenv("TOKEN_ADDRESS", "0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db"))
    # price_oracle_address: str = os.environ["PRICE_ORACLE_ADDRESS"]
    price_batch_size: int = int(os.getenv("PRICE_BATCH_SIZE","40"))
    protocol_name: Literal["velo", "aero"] = os.getenv("PROTOCOL_NAME","velo")
    good_enough_pagination_limit: int = int(os.getenv("GOOD_ENOUGH_PAGINATION_LIMIT","2000"))

    @staticmethod
    def get_config(): return SugarConfig()

    def __str__(self):
        return f"web3_provoider_uri: {self.web3_provoider_uri}\nlp_sugar_address: {self.lp_sugar_address}\ntoken_address: {self.token_address}\nprice_batch_size: {self.price_batch_size}\nprotocol_name: {self.protocol_name}\ngood_enough_pagination_limit: {self.good_enough_pagination_limit}"
    
    def __repr__(self): return str(self)


# # token we are converting from
# TOKEN_ADDRESS = normalize_address(os.environ["TOKEN_ADDRESS"])
# # token we are converting to
# STABLE_TOKEN_ADDRESS = os.environ["STABLE_TOKEN_ADDRESS"]
# # connector tokens for the pricing oracle
# # see: https://github.com/velodrome-finance/oracle
# CONNECTOR_TOKENS_ADDRESSES = list(
#     map(
#         lambda a: normalize_address(a),
#         os.environ["CONNECTOR_TOKENS_ADDRESSES"].split(","),
#     )
# )
# # how often ticker bots update
# BOT_TICKER_INTERVAL_MINUTES = int(os.environ["BOT_TICKER_INTERVAL_MINUTES"])
# # caching time for sugar tokens calls
# SUGAR_TOKENS_CACHE_MINUTES = int(os.environ["SUGAR_TOKENS_CACHE_MINUTES"])
# # caching time for sugar liquidity pools calls
# SUGAR_LPS_CACHE_MINUTES = int(os.environ["SUGAR_LPS_CACHE_MINUTES"])
# # caching time for oracle price calls
# ORACLE_PRICES_CACHE_MINUTES = int(os.environ["ORACLE_PRICES_CACHE_MINUTES"])

# # pagination limit for pools
# POOL_PAGE_SIZE = 500

# # image shown on discord embeds for pool stats
# UI_POOL_STATS_THUMBNAIL = os.environ["UI_POOL_STATS_THUMBNAIL"]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()